# Model Training
This notebook trains a machine learning model to predict house prices using the processed dataset.



In [25]:
# Import required libraries
import pandas as pd
import numpy as np
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import pickle
import os


## Load Processed Dataset
We load the processed dataset containing cleaned and engineered features.


In [26]:
# Load the processed dataset
data = pd.read_csv("../data/processed_train.csv")

# Display first few rows
print("Loaded processed dataset:")
data.head()




Loaded processed dataset:


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-1.730865,0.073375,-0.229372,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-1.728492,-0.872563,0.451936,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-1.726120,0.073375,-0.093110,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1.723747,0.309859,-0.456474,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-1.721374,0.073375,0.633618,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Define Features and Target Variable
We separate the dataset into features (`X`) and the target variable (`y`), applying log transformation to stabilize predictions.


In [ ]:
# Define target variable
target = "SalePrice"

# Apply log transformation, ensuring no zero/negative values
data = data[data[target] > 0]  # Remove rows with zero or negative SalePrice
X = data.drop(columns=[target])
y = np.log(data[target])  # Apply log transformation safely

# Drop any remaining NaN values after transformation
X = X.dropna()
y = y.dropna()

# Split dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dataset split into training and testing sets successfully.")



Dataset split into training and testing sets successfully.


c:\Users\jinno\OneDrive\Documents\vscode-projects\Predicting-Housing-Prices\.venv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Apply Feature Scaling
We scale the feature set using `StandardScaler` to improve model performance.


In [28]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit on training data and transform both train and test sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



## Train the Model
We use a `RandomForestRegressor` to train the model on the training data.


In [29]:
# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model using scaled features
model.fit(X_train_scaled, y_train)

print("Model training completed.")



ValueError: Input y contains NaN.

## Evaluate the Model
We use evaluation metrics such as Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared Score to assess model performance.


In [ ]:
# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Reverse log transformation for price predictions
y_pred_actual = np.exp(y_pred)
y_test_actual = np.exp(y_test)

# Calculate performance metrics
mae = mean_absolute_error(y_test_actual, y_pred_actual)
mse = mean_squared_error(y_test_actual, y_pred_actual)
r2 = r2_score(y_test_actual, y_pred_actual)

# Display evaluation results
print(f"Model Performance:")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared Score: {r2}")



## Save Trained Model
The trained model, feature names, and scaler are saved as `.pkl` files for future use.


In [ ]:
# Ensure the models directory exists
os.makedirs("../models", exist_ok=True)

# Save the trained model
with open("../models/trained_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save feature names
with open("../models/feature_names.pkl", "wb") as f:
    pickle.dump(X_train.columns.tolist(), f)

# Save the scaler
with open("../models/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Trained model, feature names, and scaler saved successfully!")


## Summary
- Loaded the processed dataset.
- Applied log transformation to target variable.
- Scaled feature set using `StandardScaler`.
- Trained a `RandomForestRegressor` model on scaled data.
- Evaluated model performance using MAE, MSE, and R² score.
- Saved the trained model, feature names, and scaler.
